In [1]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.WARN)
import pickle
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import os
from tensorflow.python.client import device_lib
from collections import Counter
import time

In [2]:
f = open('../../Glove/word_embedding_glove', 'rb')
word_embedding = pickle.load(f)
f.close()
word_embedding = word_embedding[: len(word_embedding)-1]

f = open('../../Glove/vocab_glove', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"

f = open("../../../dataset/sense/dict_sense-keys", 'rb')
dict_sense_keys = pickle.load(f)
f.close()

f = open("../../../dataset/sense/dict_word-sense", 'rb')
dict_word_sense = pickle.load(f)
f.close()

# Model Description
sense_word = 'support'
model_name = 'model-4-multigpu-1'
sense_word_dir = '../output/' + sense_word
model_dir = sense_word_dir + '/' + model_name
save_dir = os.path.join(model_dir, "save/")
log_dir = os.path.join(model_dir, "log")

if not os.path.exists(sense_word_dir):
    os.mkdir(sense_word_dir)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

f = open("../../../dataset/checkwords/"+ sense_word + "_data", 'rb')
data = pickle.load(f)
f.close()

data_y = []
for i in range(len(data)):
    data_y.append(dict_sense_keys[data[i][0]][3])

sense_count = Counter(data_y)
sense_count = sense_count.most_common()[:7]
vocab_sense = [k for k,v in sense_count]
vocab_sense = sorted(vocab_sense, key=lambda x:int(x[0]))
print(sense_count)
print(vocab_sense)

def make_mask_matrix(sense_word,vocab_sense):
    mask_mat = []
    sense_list = [int(string[0]) for string in vocab_sense]
    sense_count = list(Counter(sense_list).values())
    start=0
    prev=0
    for i in range(len(set(sense_list))):
        temp_row=[0]*len(sense_list)
        for j in range(len(sense_list)):
            if j>=start and j<sense_count[i]+prev:
                temp_row[j]= 0
            else:
                temp_row[j]= -10
        start+=sense_count[i]
        prev+=sense_count[i]
        mask_mat.append(temp_row)
    return mask_mat

mask_mat = make_mask_matrix(sense_word,vocab_sense)
print(mask_mat)

data_x = []
data_label = []
data_pos = []
for i in range(len(data)):
    if dict_sense_keys[data[i][0]][3] in vocab_sense:
        data_x.append(data[i][1])
        data_label.append(dict_sense_keys[data[i][0]][3])
        data_pos.append(dict_sense_keys[data[i][0]][1])


print(len(data_label), len(data_y))

# vocab_sense = dict_word_sense[sense_word]

sense2id = dict((s, i) for i,s in enumerate(vocab_sense))
id2sense = dict((i, s) for i,s in enumerate(vocab_sense))

count_pos = Counter(data_pos)
count_pos = count_pos.most_common()
vocab_pos = [int(k) for k,v in count_pos]
vocab_pos = sorted(vocab_pos, key=lambda x:int(x))

pos2id = dict((str(s), i) for i,s in enumerate(vocab_pos))
id2pos = dict((i, str(s)) for i,s in enumerate(vocab_pos))

print(vocab_pos)

max_len = 0
for i in range(len(data_x)):
    max_len = max(max_len, len(data_x[i]))
    if(len(data_x[i])>200):
        print(i)
        
print("max_len: ", max_len)

# Parameters
mode = 'train'
num_senses = len(vocab_sense)
num_pos = len(vocab_pos)
batch_size = 32
vocab_size = len(vocab)
unk_vocab_size = 1
word_emb_size = len(word_embedding[0])
max_sent_size = max(200, max_len)
hidden_size = 100
keep_prob = 0.5
l2_lambda = 0.001
init_lr = 0.005
decay_steps = 500
decay_rate = 0.96
clip_norm = 1
clipping = True
moving_avg_deacy = 0.999
num_gpus = 6
lambda_loss_pos = 5

index = []
for i in range(len(data_x)):
    index.append(i)

index_train, index_val, label_train, label_val = train_test_split(index, data_label, train_size=0.8, shuffle=True, stratify=data_label, random_state=0)

data_x = np.array(data_x)
data_pos = np.array(data_pos)

x_train = data_x[index_train]
pos_train = data_pos[index_train]
x_val = data_x[index_val]
pos_val = data_pos[index_val]

[('2:41', 1020), ('1:04', 670), ('2:32', 533), ('2:40', 503), ('2:31', 470), ('2:35', 170), ('1:21', 57)]
['1:04', '1:21', '2:41', '2:32', '2:40', '2:31', '2:35']
[[0, 0, -10, -10, -10, -10, -10], [-10, -10, 0, 0, 0, 0, 0]]
3423 3489
[1, 2]
max_len:  100


/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
        # Note that each grad_and_vars looks like the following:
        #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(grads, 0)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

In [4]:
# MODEL
tower_grads = []
losses = []
predictions = []
predictions_pos = []
total_trans_params = []

x = tf.placeholder('int32', [num_gpus, batch_size, max_sent_size], name="x")
y = tf.placeholder('int32', [num_gpus, batch_size], name="y")
y_pos = tf.placeholder('int32', [num_gpus, batch_size], name="y_pos")
x_mask  = tf.placeholder('bool', [num_gpus, batch_size, max_sent_size], name='x_mask') 
is_train = tf.placeholder('bool', [], name='is_train')
mask_matrix = tf.constant(value=mask_mat, shape=list(np.array(mask_mat).shape), dtype='float32') # mask_matrix

word_emb_mat = tf.placeholder('float', [None, word_emb_size], name='emb_mat')
input_keep_prob = tf.cond(is_train,lambda:keep_prob, lambda:tf.constant(1.0))
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(init_lr, global_step, decay_steps, decay_rate, staircase=True)

with tf.variable_scope("word_embedding"):
    unk_word_emb_mat = tf.get_variable("word_emb_mat", dtype='float', shape=[unk_vocab_size, word_emb_size], initializer=tf.contrib.layers.xavier_initializer(uniform=True, seed=0, dtype=tf.float32))
    final_word_emb_mat = tf.concat([word_emb_mat, unk_word_emb_mat], 0)

with tf.variable_scope(tf.get_variable_scope()):
    for gpu_idx in range(num_gpus):
        with tf.name_scope("model_{}".format(gpu_idx)) as scope, tf.device('/gpu:%d' % i):

            if gpu_idx > 0:
                    tf.get_variable_scope().reuse_variables()

            with tf.name_scope("word"):
                Wx = tf.nn.embedding_lookup(final_word_emb_mat, x[gpu_idx])  

            x_len = tf.reduce_sum(tf.cast(x_mask[gpu_idx], 'int32'), 1)

            with tf.variable_scope("lstm1"):
                cell_fw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw1 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw1 = tf.contrib.rnn.DropoutWrapper(cell_fw1, input_keep_prob=input_keep_prob)
                d_cell_bw1 = tf.contrib.rnn.DropoutWrapper(cell_bw1, input_keep_prob=input_keep_prob)

                (fw_h1, bw_h1), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw1, d_cell_bw1, Wx, sequence_length=x_len, dtype='float', scope='lstm1')
                h1 = tf.concat([fw_h1, bw_h1], 2)

            with tf.variable_scope("lstm2"):
                cell_fw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)
                cell_bw2 = tf.contrib.rnn.BasicLSTMCell(hidden_size,state_is_tuple=True)

                d_cell_fw2 = tf.contrib.rnn.DropoutWrapper(cell_fw2, input_keep_prob=input_keep_prob)
                d_cell_bw2 = tf.contrib.rnn.DropoutWrapper(cell_bw2, input_keep_prob=input_keep_prob)

                (fw_h2, bw_h2), _ = tf.nn.bidirectional_dynamic_rnn(d_cell_fw2, d_cell_bw2, h1, sequence_length=x_len, dtype='float', scope='lstm2')
                h = tf.concat([fw_h2, bw_h2], 2)

            def attention(input_x, input_mask, W_att):
                h_masked = tf.boolean_mask(input_x, input_mask)
                h_tanh = tf.tanh(h_masked)
                u = tf.matmul(h_tanh, W_att)
                a = tf.nn.softmax(u)
                c = tf.reduce_sum(tf.multiply(h_tanh, a), 0)  
                return c

            with tf.variable_scope("attention"):
                W_att = tf.get_variable("W_att", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                c = tf.expand_dims(attention(h[0], x_mask[gpu_idx][0], W_att), 0)
                for i in range(1, batch_size):
                    c = tf.concat([c, tf.expand_dims(attention(h[i], x_mask[gpu_idx][i], W_att), 0)], 0)
           
            with tf.variable_scope("attention_pos"):
                W_attp = tf.get_variable("W_attp", shape=[2*hidden_size, 1], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                cp = tf.expand_dims(attention(h[0], x_mask[gpu_idx][0], W_attp), 0)
                for i in range(1, batch_size):
                    cp = tf.concat([cp, tf.expand_dims(attention(h1[i], x_mask[gpu_idx][i], W_attp), 0)], 0)        
            
            with tf.variable_scope("softmax_layer_pos"):
                Wp = tf.get_variable("Wp", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                bp = tf.get_variable("bp", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_cp = tf.nn.dropout(cp, input_keep_prob)
                logits_pos = tf.matmul(drop_cp, Wp) + bp
                prediction_pos = tf.argmax(logits_pos, 1)
                predictions_pos.append(prediction_pos)
                final_masking = tf.nn.embedding_lookup(mask_matrix, prediction_pos)
            
            with tf.variable_scope("softmax_layer"):
                W = tf.get_variable("W", shape=[2*hidden_size, num_senses], initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.1, seed=0))
                b = tf.get_variable("b", shape=[num_senses], initializer=tf.zeros_initializer())
                drop_c = tf.nn.dropout(c, input_keep_prob)
                logits = tf.matmul(drop_c, W) + b
                masked_logits = logits + final_masking
                predictions.append(tf.argmax(masked_logits, 1))
            
            loss_pos = lambda_loss_pos * tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits_pos, labels=y_pos[gpu_idx]))
            loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=masked_logits, labels=y[gpu_idx]))
            l2_loss = l2_lambda * tf.losses.get_regularization_loss()

            total_loss = loss + l2_loss + loss_pos
            tf.summary.scalar("loss_{}".format(gpu_idx), total_loss)

            summaries = tf.get_collection(tf.GraphKeys.SUMMARIES, scope)

            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_vars = optimizer.compute_gradients(total_loss)

            clipped_grads = grads_vars
            if(clipping == True):
                clipped_grads = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in clipped_grads]

            tower_grads.append(clipped_grads)
            losses.append(total_loss)

tower_grads = average_gradients(tower_grads)
losses = tf.add_n(losses)/len(losses)
apply_grad_op = optimizer.apply_gradients(tower_grads, global_step=global_step)
summaries.append(tf.summary.scalar('total_loss', losses))
summaries.append(tf.summary.scalar('learning_rate', learning_rate))

for var in tf.trainable_variables():
    summaries.append(tf.summary.histogram(var.op.name, var))

variable_averages = tf.train.ExponentialMovingAverage(moving_avg_deacy, global_step)
variables_averages_op = variable_averages.apply(tf.trainable_variables())

train_op = tf.group(apply_grad_op, variables_averages_op)
saver = tf.train.Saver(tf.global_variables())
summary = tf.summary.merge(summaries)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# print (device_lib.list_local_devices())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())                          # For initializing all the variables
summary_writer = tf.summary.FileWriter(log_dir, sess.graph)          # For writing Summaries

In [6]:
def data_prepare(x, y, p):
    num_examples = len(x)

    xx = np.zeros([num_examples, max_sent_size], dtype=int)
    xx_mask = np.zeros([num_examples, max_sent_size], dtype=bool)
    yy = np.zeros([num_examples], dtype=int)
    pp = np.zeros([num_examples], dtype=int)
    
    for j in range(num_examples):
        for i in range(max_sent_size):
            if(i>=len(x[j])):
                break
            w = x[j][i]
            xx[j][i] = word2id[w] if w in word2id else word2id['UNKNOWN_TOKEN']
            xx_mask[j][i] = True
        yy[j] = sense2id[y[j]]
        pp[j] = pos2id[p[j]]
    return xx, xx_mask, yy, pp

def model(xx, yy, mask, pp, train_cond=True):
    num_batches = int(len(xx)/(batch_size*num_gpus))
    _losses = 0
    preds = []
    preds_pos = []
    
    for j in range(num_batches): 
        
        s = j * batch_size * num_gpus
        e = (j+1) * batch_size * num_gpus
        xx_re = xx[s:e].reshape([num_gpus, batch_size, -1])
        yy_re = yy[s:e].reshape([num_gpus, batch_size])
        pp_re = pp[s:e].reshape([num_gpus, batch_size])
        mask_re = mask[s:e].reshape([num_gpus, batch_size, -1])
        
        feed_dict = {x:xx_re, y:yy_re, y_pos:pp_re, x_mask:mask_re, is_train:train_cond, input_keep_prob:keep_prob, word_emb_mat:word_embedding}
        
        if(train_cond==True):
            _, _loss, step, _summary = sess.run([train_op, losses, global_step, summary], feed_dict)
            summary_writer.add_summary(_summary, step)
#             print("Steps:{}".format(step), ", Loss: {}".format(_loss))

        else:
            _loss, pred, pred_pos = sess.run([losses, predictions, predictions_pos], feed_dict)
            for i in range(num_gpus):
                preds.append(pred[i])  
                preds_pos.append(pred_pos[i])
                
        _losses +=_loss

    if(train_cond==False):
        y_pred = []
        pos_pred = []
        for pred in preds:
            for bt in pred:
                y_pred.append(bt)
        
        for pred in preds_pos:
            for bt in pred:
                pos_pred.append(bt)     

        return _losses/num_batches, y_pred, pos_pred
    
    return _losses/num_batches, step

def eval_score(yy, pred, pp, pred_pos):
    num_batches = int(len(yy)/(batch_size*num_gpus))
    f1 = f1_score(yy[:batch_size*num_batches*num_gpus], pred, average='macro')
    f1_pos = f1_score(pp[:batch_size*num_batches*num_gpus], pred_pos, average='macro')
    accu = accuracy_score(yy[:batch_size*num_batches*num_gpus], pred)
    accu_pos = accuracy_score(pp[:batch_size*num_batches*num_gpus], pred_pos)
    return f1*100, accu*100, f1_pos*100, accu_pos*100

In [7]:
x_id_train, mask_train, y_train, pos_id_train = data_prepare(x_train, label_train, pos_train)
x_id_val, mask_val, y_val, pos_id_val = data_prepare(x_val, label_val,pos_val)

In [9]:
num_epochs = 60
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]

    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, mask_train, pos_id_train)
    time_taken = time.time() - start_time
    print("Epoch:", i+1,"Step:", step, "loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        start_time = time.time()
        train_loss, train_pred, train_pred_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, pos_id_train, train_pred_pos)
        time_taken = time.time() - start_time
        print("Train: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), " POS: F1 Score:{0:.4f}".format(f1_pos_), "Accuracy:{0:.4f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
        
        start_time = time.time()
        val_loss, val_pred, val_pred_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, pos_id_val, val_pred_pos)
        time_taken = time.time() - start_time
        print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), " POS: F1 Score:{0:.4f}".format(f1_pos_), "Accuracy:{0:.4f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))
        

Epoch: 1 Step: 14 loss:29.0664 , Time: 113.9897
Epoch: 2 Step: 28 loss:6.0515 , Time: 109.6459
Epoch: 3 Step: 42 loss:4.5205 , Time: 109.5390
Epoch: 4 Step: 56 loss:3.7873 , Time: 111.0478
Epoch: 5 Step: 70 loss:3.5367 , Time: 109.5774
Model Saved
Train: F1 Score:44.1629 Accuracy:54.2411  POS: F1 Score:79.7831 Accuracy:88.5789 Loss:3.8131 , Time: 76.3288


/users/btech/aviraj/cs771/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Val: F1 Score:42.0617 Accuracy:52.6042  POS: F1 Score:51.2547 Accuracy:87.5000 Loss:4.3173 , Time: 16.4599
Epoch: 6 Step: 84 loss:3.6355 , Time: 110.2638
Epoch: 7 Step: 98 loss:3.1059 , Time: 110.6149
Epoch: 8 Step: 112 loss:3.2361 , Time: 109.6674
Epoch: 9 Step: 126 loss:2.6152 , Time: 110.9451
Epoch: 10 Step: 140 loss:2.3518 , Time: 109.1211
Model Saved
Train: F1 Score:53.3859 Accuracy:63.2812  POS: F1 Score:90.9026 Accuracy:93.8616 Loss:2.2108 , Time: 74.2965
Val: F1 Score:54.2697 Accuracy:60.9375  POS: F1 Score:90.4621 Accuracy:93.9236 Loss:2.2183 , Time: 16.1303
Epoch: 11 Step: 154 loss:2.3001 , Time: 108.7076
Epoch: 12 Step: 168 loss:2.0606 , Time: 107.3658
Epoch: 13 Step: 182 loss:1.7126 , Time: 108.3604
Epoch: 14 Step: 196 loss:1.6052 , Time: 106.9204
Epoch: 15 Step: 210 loss:1.4871 , Time: 106.2940
Model Saved
Train: F1 Score:59.5127 Accuracy:69.7545  POS: F1 Score:95.7824 Accuracy:97.2470 Loss:1.3217 , Time: 74.3912
Val: F1 Score:55.7068 Accuracy:65.6250  POS: F1 Score:93.394

In [9]:
num_epochs = 20
log_period = 5

for i in range(num_epochs):
    random = np.random.choice(len(y_train), size=(len(y_train)), replace=False)
    x_id_train = x_id_train[random]
    y_train = y_train[random]
    mask_train = mask_train[random]    
    pos_id_train = pos_id_train[random]

    start_time = time.time()
    train_loss, step = model(x_id_train, y_train, mask_train, pos_id_train)
    time_taken = time.time() - start_time
    print("Epoch:", i+1,"Step:", step, "loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
    
    if((i+1)%log_period==0):
        saver.save(sess, save_path=save_dir)                         
        print("Model Saved")
        start_time = time.time()
        train_loss, train_pred, train_pred_pos = model(x_id_train, y_train, mask_train, pos_id_train, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_train, train_pred, pos_id_train, train_pred_pos)
        time_taken = time.time() - start_time
        print("Train: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), " POS: F1 Score:{0:.4f}".format(f1_pos_), "Accuracy:{0:.4f}".format(accu_pos_), "Loss:{0:.4f}".format(train_loss), ", Time: {0:.4f}".format(time_taken))
        
        start_time = time.time()
        val_loss, val_pred, val_pred_pos = model(x_id_val, y_val, mask_val, pos_id_val, train_cond=False)        
        f1_, accu_, f1_pos_, accu_pos_ = eval_score(y_val, val_pred, pos_id_val, val_pred_pos)
        time_taken = time.time() - start_time
        print("Val: F1 Score:{0:.4f}".format(f1_), "Accuracy:{0:.4f}".format(accu_), " POS: F1 Score:{0:.4f}".format(f1_pos_), "Accuracy:{0:.4f}".format(accu_pos_), "Loss:{0:.4f}".format(val_loss), ", Time: {0:.4f}".format(time_taken))
        

Epoch: 1 Step: 854 loss:0.3853 , Time: 128.9813
Epoch: 2 Step: 868 loss:0.4631 , Time: 116.8871
Epoch: 3 Step: 882 loss:0.3826 , Time: 116.2631
Epoch: 4 Step: 896 loss:0.4696 , Time: 117.0492
Epoch: 5 Step: 910 loss:0.4449 , Time: 112.1934
Model Saved
Train: F1 Score:89.2361 Accuracy:89.3229  POS: F1 Score:99.4427 Accuracy:99.6280 Loss:0.3708 , Time: 78.3031
Val: F1 Score:70.3164 Accuracy:71.0069  POS: F1 Score:96.5030 Accuracy:97.7431 Loss:2.6667 , Time: 16.6270
Epoch: 6 Step: 924 loss:0.3920 , Time: 112.3937
Epoch: 7 Step: 938 loss:0.3927 , Time: 114.0774
Epoch: 8 Step: 952 loss:0.3503 , Time: 113.8995
Epoch: 9 Step: 966 loss:0.3349 , Time: 113.5894
Epoch: 10 Step: 980 loss:0.3155 , Time: 114.0529
Model Saved
Train: F1 Score:91.6966 Accuracy:90.3646  POS: F1 Score:99.5000 Accuracy:99.6652 Loss:0.3319 , Time: 78.0264
Val: F1 Score:68.6777 Accuracy:69.0972  POS: F1 Score:96.5470 Accuracy:97.7431 Loss:2.7834 , Time: 16.5779
Epoch: 11 Step: 994 loss:0.2867 , Time: 115.8982
Epoch: 12 Step

In [8]:
saver.restore(sess, save_dir)